In [1]:
import pandas as pd
from pathlib import Path
from sklearn import tree
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
file_path = Path("./Resources/Churn_Modelling_2.csv")
df = pd.read_csv(file_path)
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# Drop unnecessary columns 
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
df.nunique()

CreditScore         460
Geography             3
Gender                2
Age                  70
Tenure               11
Balance            6382
NumOfProducts         4
HasCrCard             2
IsActiveMember        2
EstimatedSalary    9999
Exited                2
dtype: int64

In [16]:
# Binary encoding
encoded = pd.get_dummies(df, columns=["Gender", "Geography"])
encoded.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0,0,0,1


In [17]:
# Define features set
X = encoded.copy()
X = X.drop("Exited", axis=1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,1,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,1,0,0,0,1


In [18]:
# Define target vector
y = encoded["Exited"].values.reshape(-1, 1)
y[:5]

array([[1],
       [0],
       [1],
       [0],
       [0]])

In [19]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [20]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7500, 13)
(2500, 13)
(7500, 1)
(2500, 1)


In [23]:
# Creating StandardScaler instance
scaler = StandardScaler()

In [24]:
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)

In [25]:
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [26]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [27]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [28]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [29]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [30]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1728,253
Actual 1,273,246


Accuracy Score : 0.7896
Classification Report
              precision    recall  f1-score   support

           0       0.86      0.87      0.87      1981
           1       0.49      0.47      0.48       519

    accuracy                           0.79      2500
   macro avg       0.68      0.67      0.68      2500
weighted avg       0.79      0.79      0.79      2500



In [32]:
# MODEL WITH AGE OUTLIERS REMOVED

In [9]:
file_path = Path("./Resources/a_out_df.csv")
a_out_df = pd.read_csv(file_path)
a_out_df.head()

,Unnamed: 0,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,gender
0,0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0
1,1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0
2,2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0
3,3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0
4,4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0


In [10]:
a_out_df.dtypes

Unnamed: 0           int64
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
gender               int64
dtype: object

In [11]:
# Binary encoding
a_out_encoded = pd.get_dummies(a_out_df, columns=["Gender", "Geography"])
a_out_encoded.head()

,Unnamed: 0,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,gender,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,0,619,42,2,0.00,1,1,1,101348.88,1,0,1,0,1,0,0
1,1,608,41,1,83807.86,1,0,1,112542.58,0,0,1,0,0,0,1
2,2,502,42,8,159660.80,3,1,0,113931.57,1,0,1,0,1,0,0
3,3,699,39,1,0.00,2,0,0,93826.63,0,0,1,0,1,0,0
4,4,850,43,2,125510.82,1,1,1,79084.10,0,0,1,0,0,0,1


In [13]:
# Drop unnecessary columns 
a_out_df = a_out_df.drop(['Unnamed: 0', 'gender'], axis=1)
a_out_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9584,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9585,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9586,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9587,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [15]:
# Define features set
X = a_out_encoded.copy()
X = X.drop("Exited", axis=1)
X.head()

,Unnamed: 0,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,gender,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,0,619,42,2,0.00,1,1,1,101348.88,0,1,0,1,0,0
1,1,608,41,1,83807.86,1,0,1,112542.58,0,1,0,0,0,1
2,2,502,42,8,159660.80,3,1,0,113931.57,0,1,0,1,0,0
3,3,699,39,1,0.00,2,0,0,93826.63,0,1,0,1,0,0
4,4,850,43,2,125510.82,1,1,1,79084.10,0,1,0,0,0,1


In [16]:
# Define target vector
y = a_out_encoded["Exited"].values.reshape(-1, 1)
y[:5]

array([[1],
       [0],
       [1],
       [0],
       [0]])

In [17]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [18]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7191, 15)
(2398, 15)
(7191, 1)
(2398, 1)


In [19]:
# Creating StandardScaler instance
scaler = StandardScaler()
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [20]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [21]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [22]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [23]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [24]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1663,237
Actual 1,267,231


Accuracy Score : 0.7898248540450375
Classification Report
              precision    recall  f1-score   support

           0       0.86      0.88      0.87      1900
           1       0.49      0.46      0.48       498

    accuracy                           0.79      2398
   macro avg       0.68      0.67      0.67      2398
weighted avg       0.79      0.79      0.79      2398



In [25]:
## MODEL WITH CREDIT SCORE OUTLIERS REMOVED

In [32]:
file_path = Path("./Resources/C_out_df.csv")
c_out_df = pd.read_csv(file_path)
c_out_df.head()

,Unnamed: 0,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,gender
0,0,619,France,Female,42,2,0.00,1,1,1,101348.88,1,0
1,1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,0
2,2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1,0
3,3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0
4,4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,0


In [33]:
c_out_df.dtypes

Unnamed: 0           int64
CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
gender               int64
dtype: object

In [34]:
# Drop unnecessary columns 
c_out_df = c_out_df.drop(['Unnamed: 0', 'gender'], axis=1)
c_out_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9979,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9980,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9981,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9982,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [35]:
# Binary encoding
c_out_encoded = pd.get_dummies(c_out_df, columns=["Gender", "Geography"])
c_out_encoded.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0,0,0,1


In [36]:
# Define features set
X = c_out_encoded.copy()
X = X.drop("Exited", axis=1)
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Gender_Female,Gender_Male,Geography_France,Geography_Germany,Geography_Spain
0,619,42,2,0.00,1,1,1,101348.88,1,0,1,0,0
1,608,41,1,83807.86,1,0,1,112542.58,1,0,0,0,1
2,502,42,8,159660.80,3,1,0,113931.57,1,0,1,0,0
3,699,39,1,0.00,2,0,0,93826.63,1,0,1,0,0
4,850,43,2,125510.82,1,1,1,79084.10,1,0,0,0,1


In [37]:
# Define target vector
y = c_out_encoded["Exited"].values.reshape(-1, 1)
y[:5]

array([[1],
       [0],
       [1],
       [0],
       [0]])

In [38]:
# Splitting into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [39]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(7488, 13)
(2496, 13)
(7488, 1)
(2496, 1)


In [40]:
# Creating StandardScaler instance
scaler = StandardScaler()
# Fitting Standard Scaller
X_scaler = scaler.fit(X_train)
# Scaling data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [41]:
# Creating the decision tree classifier instance
model = tree.DecisionTreeClassifier()

In [42]:
# Fitting the model
model = model.fit(X_train_scaled, y_train)

In [43]:
# Making predictions using the testing data
predictions = model.predict(X_test_scaled)

In [44]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions)

In [47]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions))


Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,1713,294
Actual 1,231,258


Accuracy Score : 0.7896634615384616
Classification Report
              precision    recall  f1-score   support

           0       0.88      0.85      0.87      2007
           1       0.47      0.53      0.50       489

    accuracy                           0.79      2496
   macro avg       0.67      0.69      0.68      2496
weighted avg       0.80      0.79      0.79      2496

